# 14) Linear Algebra


## Today

 1. Matrices as linear transformations  
 2. Polynomial evaluation and fitting  
 3. Orthogonality

In [1]:
using Plots
default(linewidth=4, legendfontsize=12)

## 1. Matrices as linear transformations

:::{tip}
Resource: An excellent resource for Linear Algebra is the MIT OpenCourseWare course by Prof. Gilbert Strang (now retired). Here is a link to the [Fall 2011](https://ocw.mit.edu/courses/18-06sc-linear-algebra-fall-2011/download/) undergraduate course edition. Other [video galleries](https://ocw.mit.edu/courses/18-06-linear-algebra-spring-2010/video_galleries/video-lectures/) of his lectures can also be found.
:::

Linear algebra is the study of linear transformations on vectors, which represent points in a finite dimensional space.  The matrix-vector product $y = A x$ is a linear combination of the columns of $A$.  The familiar definition,

$$ y_i = \sum_j A_{i,j} x_j $$

can also be viewed as

$$ y = \Bigg[ A_{:,1} \Bigg| A_{:,2} \Bigg| \dotsm \Bigg] \begin{bmatrix} x_1 \\ x_2 \\ \vdots \end{bmatrix}
= \Bigg[ A_{:,1} \Bigg] x_1 + \Bigg[ A_{:,2} \Bigg] x_2 + \dotsb . $$

That is, a linear combination of the columns of A.

### Math and Julia Notation

The notation $A_{i,j}$ corresponds to the Julia syntax `A[i,j]` and the colon `:` means the entire range (row or column).  So $A_{:,j}$ is the $j$th column and $A_{i,:}$ is the $i$th row.  The corresponding Julia syntax is `A[:,j]` and `A[i,:]`.

Julia has syntax for row vectors, column vectors, and arrays.

In [ ]:
[1 2 3] # a row vector

In [ ]:
[1; 2; 3] # a column vector

In [ ]:
[1. 2 3; 4 5 6] # a 2x3 real matrix
# compare with Python's syntax: np.array([[1, 2, 3], [4, 5, 6]])

In [ ]:
[1 2; 4 3]

In [ ]:
[1 0; 0 2; 10 3]

In [ ]:
[1; 2 + 1im; 3]' # ' is transpose, and for complex-valued matrices is the conjugate transpose ("adjoint")

### Implementing multiplication by row

In [ ]:
function matmult1(A, x)
    m, n = size(A)
    y = zeros(m)
    for i in 1:m # row index first, i.e., for each row
        for j in 1:n # we iterate over the columns
            y[i] += A[i,j] * x[j] # we apply the familiar definition
        end
    end
    y
end


In [ ]:
A = reshape(1.:12, 3, 4) # a 3x4 matrix with the numbers 1:12

In [ ]:
x = [10., 0, 0, 0]

In [ ]:
matmult1(A, x)

In [ ]:
# Dot product
A[2, :]' * x

In [ ]:
function matmult2(A, x)
    m, n = size(A)
    y = zeros(m)
    for i in 1:m # iterate over rows of A
        y[i] = A[i,:]' * x # this way we use the dot product between the transposed row of A and the whole x
    end
    y
end

matmult2(A, x)

Curiosity: which one is faster? Which one takes more allocations?


In [ ]:
@time matmult1(A,x)
@time matmult2(A,x)

### Implementing multiplication by column


In [ ]:
function matmult3(A, x)
    m, n = size(A)
    y = zeros(m)
    for j in 1:n # iterate over columns of A
        y += A[:, j] * x[j]
    end
    y
end

matmult3(A, x)

In [ ]:
@time matmult3(A, x)

In [ ]:
A * x # built-in matrix-vector multiply

In [ ]:
@time A * x  # We'll use this version

Check the standard operations [documentation page](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#Standard-functions).

## 2. Polynomial evaluation 
### Polynomial evaluation is (continuous) linear algebra
We can evaluate polynomials using matrix-vector multiplication.
For example,
$$ - 3x + 5x^3 = \Bigg[ 1 \Bigg|\, x \Bigg|\, x^2 \,\Bigg|\, x^3 \Bigg] \begin{bmatrix}0 \\ -3 \\ 0 \\ 5 \end{bmatrix} . $$

In [ ]:
using Pkg
Pkg.add("Polynomials")

using Polynomials
P(x) = Polynomial(x)

p = [0, -3, 0, 5] # vector of coefficients for the canonical basis
q = [1, 2, 3, 4]
@show f = P(p)
@show g = P(q)
h = f + g
@show h
@show P(p+q) # we can see that the polynomial evaluation as a linear combination (of matrix columns) is a linear transformation!
x = [0., 1, 2]
h.(x) # can be applied to each element of a vector with the dot operator

In [ ]:
plot(h, legend=:bottomright, xlim=(-2, 2))

### Polynomial evaluation is (discrete) linear algebra

In [ ]:
V = [one.(x) x x.^2 x.^3] # Vandermonde matrix

In [ ]:
V * p + V * q # same as h.(x)

In [ ]:
V * (p + q) # again, matrix multiplication is a linear transformation

### Vandermonde matrices

A [Vandermonde matrix](https://en.wikipedia.org/wiki/Vandermonde_matrix) is one whose columns are polynomials (monomials) evaluated at discrete points.

$$V(x) = \begin{bmatrix} 1 \Bigg| x \Bigg| x^2 \Bigg| x^3 \Bigg| \dotsb \end{bmatrix}$$

In [ ]:
function vander(x, k=nothing)
    if isnothing(k)
        k = length(x)
    end
    m = length(x)
    V = ones(m, k)
    for j in 2:k
        V[:, j] = V[:, j-1] .* x
    end
    V
end

In [ ]:
@show x = LinRange(-1, 1, 50)
V = vander(x, 4)
scatter(x, V, legend=:bottomright, label = ["V_1" "V_2" "V_3" "V_4"])

### Fitting (polynomial interpolation) is linear algebra

$$ \underbrace{\begin{bmatrix} 1 \Bigg| x \Bigg| x^2 \Bigg| x^3 \Bigg| \dotsb \end{bmatrix}}_{V(x)} \Big[ p \Big] = \Bigg[ y \Bigg]$$

In [ ]:
x1 = [-.9, 0.1, .5, .8]
y1 = [1, 2.4, -.2, 1.3]
scatter(x1, y1, markersize=8)

In [ ]:
V = vander(x1)
@show size(V)
p = V \ y1 # write y1 in the polynomial basis; left-division, read V^{-1} * y1 (like solving p V = y1)
scatter(x1, y1, markersize=8, xlims=(-1, 1))
# plot!(P(p), label="P(p)")
plot!(x, vander(x, 4) * p, label="\$ V(x) p\$", linestyle=:dash)

### Some common terminology

* The **range** of $A$ is the space spanned by its columns.  This definition coincides with the range of a function $f(x)$ when $f(x) = A x$.
* The (right) **nullspace** of $A$ is the space of vectors $x$ such that $A x = 0$.
* The **rank** of $A$ is the dimension of its range.
* A matrix has **full rank** if the nullspace of either $A$ or $A^T$ is empty (only the 0 vector).  Equivalently, if all the columns of $A$ (or $A^T$) are linearly independent.
* A **nonsingular** (or **invertible**) matrix is a square matrix of full rank.  We call the inverse $A^{-1}$ and it satisfies $A^{-1} A = A A^{-1} = I$.

$\DeclareMathOperator{\rank}{rank} \DeclareMathOperator{\null}{null} $

#### [Poll 15.1](https://www.polleverywhere.com/multiple_choice_polls/WUil3iBOYpi6MfryOa5OT): If $A \in \mathbb{R}^{m\times m}$, which of these doesn't belong?

1. $A$ has an inverse, $A^{-1}$
2. $\rank (A) = m$
3. $\null(A) = \{0\}$
4. $A A^T = A^T A$
5. $\det(A) \ne 0$
6. $A x = 0$ implies that $x = 0$

In [ ]:
Pkg.add("LinearAlgebra")
using LinearAlgebra

A = rand(4, 4)
B = A' * A - A * A' # no. 4! (only valid for orthogonal matrices - more later)
@show B
det(A)

### What is an inverse?

When we write $x = A^{-1} y$, we mean that $x$ is the unique vector such that $A x = y$.
(It is rare that we explicitly compute a matrix $A^{-1}$, though [it's not as "bad"](https://arxiv.org/abs/1201.6035) as people may have told you.)
A vector $y$ is equivalent to $\sum_i e_i y_i$ where $e_i$ are columns of the identity.
Meanwhile, $x = A^{-1} y$ means that we are expressing that same vector $y$ in the basis of the columns of $A$, i.e., $\sum_i A_{:,i} x_i$.


In [ ]:
using LinearAlgebra
A = rand(4, 4)

In [ ]:
A \ A # left-division, read A^{-1} * A; notice I (all 1's on the diagonal, and numerically 0's elsewhere)

In [ ]:
inv(A) * A

## 3. Orthogonality

The **inner product**
$$ x^T y = \sum_i x_i y_i $$
of real vectors (or columns of a matrix) tells us about their magnitude and about the angle.
The **norm** is induced by the inner product,
$$ \lVert x \rVert = \sqrt{x^T x} $$
and the angle $\theta$ is defined by
$$ \cos \theta = \frac{x^T y}{\lVert x \rVert \, \lVert y \rVert} . $$
Inner products are **bilinear**, which means that they satisfy some convenient algebraic properties
$$ \begin{split}
(x + y)^T z &= x^T z + y^T z \\
x^T (y + z) &= x^T y + x^T z \\
(\alpha x)^T (\beta y) &= \alpha \beta x^T y \\
\end{split} . $$

### Examples with inner products

In [ ]:
x = [0, 1]
y = [1, 1]
@show x' * y
@show y' * x;

In [ ]:
ϕ = pi/6
y = [cos(ϕ), sin(ϕ)]
cos_θ = x'*y / (norm(x) * norm(y))
@show cos_θ
@show cos(ϕ-pi/2);

### Polynomials can be orthogonal too!


In [ ]:
x = LinRange(-1, 1, 50)
A = vander(x, 4)
M = A * [.5 0 0 0; # 0.5
         0  1 0 0;  # x
         0  0 1 0]' # x^2
# that is, M = [0.5 | x | x^2]
scatter(x, M, label = ["M_1" "M_2" "M_3"])
plot!(x, 0*x, label=:none, color=:black)

* Which inner product will be zero?

  * Which functions are even and odd?

### Polynomial inner products

In [ ]:
M[:,1]' * M[:,2]

In [ ]:
M[:,1]' * M[:,3]

In [ ]:
M[:,2]' * M[:,3]